## Create data

In [17]:
import os
import json

def save_data(data, type):
    # save your preprocessed data
    with open(os.path.join("data/", type + ".json"), "w") as f:
        json.dump(data, f, indent= 4)
    return

In [18]:
import json

with open("data/train.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)
f.close()

from sklearn.model_selection import train_test_split
train_data, valid_data  = train_test_split(train_data["data"], test_size= 0.1)

In [19]:
train_data = {"data": train_data}
valid_data = {"data": valid_data}

save_data(train_data, "new_train")
save_data(valid_data, "valid")

## Load data & Train model

In [8]:
from datasets import load_dataset

raw_datasets = load_dataset("visquad.py")

Reusing dataset visquad (C:\Users\phkha\.cache\huggingface\datasets\visquad\plain_text\1.0.0\33e7735eec1e054b40673181eb3e6cbaf495119662fa6e6a993f5301d477abc1)
100%|██████████| 2/2 [00:00<00:00, 222.19it/s]


In [9]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 26418
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2039
    })
})


In [3]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  Động vật lưỡng cư (danh pháp khoa học: Amphibia) là một lớp động vật có xương sống máu lạnh. Tất cả các loài lưỡng cư hiện đại đều là phân nhánh Lissamphibia của nhóm lớn Amphibia này. Động vật lưỡng cư phải trải qua quá trình biến thái từ ấu trùng sống dưới nước tới dạng trưởng thành có phổi thở không khí, mặc dù vài loài đã phát triển qua nhiều giai đoạn khác nhau để bảo vệ hoặc bỏ qua giai đoạn ấu trùng ở trong nước dễ gặp nguy hiểm. Da được dùng như cơ quan hô hấp phụ, một số loài kỳ giông và ếch thiếu phổi phụ thuộc hoàn toàn vào da. Động vật lưỡng cư có hình dáng giống bò sát, nhưng bò sát, cùng với chim và động vật có vú, là các loài động vật có màng ối và không cần có nước để sinh sản. Trong những thập kỷ gần đây, đã có sự suy giảm số lượng của nhiều loài lưỡng cư trên toàn cầu.
Question:  Tên khoa học của động vật lưỡng cư là gì?
Answer:  {'text': ['Amphibia'], 'answer_start': [39]}


During training, there is only **ONE** possible answer. We have to filter datasets which has only **ONE** possible answer.

In [10]:
raw_datasets["train"] = raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) == 1)

Loading cached processed dataset at C:\Users\phkha\.cache\huggingface\datasets\visquad\plain_text\1.0.0\33e7735eec1e054b40673181eb3e6cbaf495119662fa6e6a993f5301d477abc1\cache-37d2960dc868c755.arrow


In [11]:
raw_datasets["validation"] = raw_datasets["validation"].filter(lambda x: len(x["answers"]["text"]) == 1)

Loading cached processed dataset at C:\Users\phkha\.cache\huggingface\datasets\visquad\plain_text\1.0.0\33e7735eec1e054b40673181eb3e6cbaf495119662fa6e6a993f5301d477abc1\cache-3c0aadcd14a87cac.arrow


In [12]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 17825
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1415
    })
})

In [1]:
from tqdm import tqdm
from time import sleep
 
 
for i in tqdm(range(0, 100), total = 500,
              desc ="Text You Want"):
    sleep(.1)

Text You Want:  20%|██        | 100/500 [00:10<00:43,  9.19it/s]


git clone --single-branch --branch fast_tokenizers_BARTpho_PhoBERT_BERTweet https://github.com/datquocnguyen/transformers.git

cd transformers

pip3 install -e .

In [13]:
import torch
from transformers import BartphoTokenizerFast

tokenizer = BartphoTokenizerFast.from_pretrained("vinai/bartpho-syllable")
tokenizer.is_fast

True

or you can use:
```(python)
import torch
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("vinai/bartpho-syllable")
tokenizer.is_fast
```

In [14]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'<s> Tên khoa học của động vật lưỡng cư là gì?</s></s> Động vật lưỡng cư (danh pháp khoa học: Amphibia) là một lớp động vật có xương sống máu lạnh. Tất cả các loài lưỡng cư hiện đại đều là phân nhánh Lissamphibia của nhóm lớn Amphibia này. Động vật lưỡng cư phải trải qua quá trình biến thái từ ấu trùng sống dưới nước tới dạng trưởng thành có phổi thở không khí, mặc dù vài loài đã phát triển qua nhiều giai đoạn khác nhau để bảo vệ hoặc bỏ qua giai đoạn ấu trùng ở trong nước dễ gặp nguy hiểm. Da được dùng như cơ quan hô hấp phụ, một số loài kỳ giông và ếch thiếu phổi phụ thuộc hoàn toàn vào da. Động vật lưỡng cư có hình dáng giống bò sát, nhưng bò sát, cùng với chim và động vật có vú, là các loài động vật có màng ối và không cần có nước để sinh sản. Trong những thập kỷ gần đây, đã có sự suy giảm số lượng của nhiều loài lưỡng cư trên toàn cầu.</s>'

In [15]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))
    print()

<s> Tên khoa học của động vật lưỡng cư là gì?</s></s> Động vật lưỡng cư (danh pháp khoa học: Amphibia) là một lớp động vật có xương sống máu lạnh. Tất cả các loài lưỡng cư hiện đại đều là phân nhánh Lissamphibia của nhóm lớn Amphibia này. Động vật lưỡng cư phải trải qua quá trình biến thái từ ấu trùng sống dưới nước tới dạng trưởng thành có phổi thở không khí,</s>

<s> Tên khoa học của động vật lưỡng cư là gì?</s></s> hiện đại đều là phân nhánh Lissamphibia của nhóm lớn Amphibia này. Động vật lưỡng cư phải trải qua quá trình biến thái từ ấu trùng sống dưới nước tới dạng trưởng thành có phổi thở không khí, mặc dù vài loài đã phát triển qua nhiều giai đoạn khác nhau để bảo vệ hoặc bỏ qua giai đoạn ấu trùng ở trong nước dễ gặp nguy hiểm. Da</s>

<s> Tên khoa học của động vật lưỡng cư là gì?</s></s>u trùng sống dưới nước tới dạng trưởng thành có phổi thở không khí, mặc dù vài loài đã phát triển qua nhiều giai đoạn khác nhau để bảo vệ hoặc bỏ qua giai đoạn ấu trùng ở trong nước dễ gặp nguy 

`max_length` to set the maximum length (here 100)

`truncation="only_second"` to truncate the context (which is in the second position) when the question with its context is too long

`stride` to set the number of overlapping tokens between two successive chunks (here 50)

`return_overflowing_tokens=True` to let the tokenizer know we want the overflowing tokens

This is how `stride` works:

In [16]:
a = " hiện đại đều là phân nhánh Lissamphibia của nhóm lớn Amphibia này. Động vật lưỡng cư phải trải qua quá trình biến thái từ ấu trùng sống dưới nước tới dạng trưởng thành có phổi thở không khí,"
c = tokenizer.tokenize(a)
print(len(c))

50


In [17]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [18]:
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0, 0]

In [24]:
inputs = tokenizer(
    raw_datasets["train"][2:5]["question"],
    raw_datasets["train"][2:5]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 3 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 3 examples gave 18 features.
Here is where each comes from: [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2].


In [25]:
raw_datasets["train"][2:5]["answers"]

[{'text': ['dưới nước'], 'answer_start': [254]},
 {'text': ['phổi'], 'answer_start': [289]},
 {'text': ['có màng ối và không cần có nước để sinh sản'],
  'answer_start': [658]}]

In [27]:
inputs

{'input_ids': [[0, 2104, 40, 78, 274, 7, 23, 178, 107, 1749, 171, 1694, 702, 168, 642, 9, 27, 995, 286, 2, 2, 2483, 405, 171, 1694, 702, 24, 3904, 140, 267, 465, 71, 32, 4016, 3306, 4474, 73, 9, 14, 830, 56, 405, 8, 2097, 274, 1115, 1376, 5, 1809, 110, 10, 745, 171, 1694, 702, 47, 257, 344, 9, 531, 2215, 1647, 345, 8625, 3306, 4474, 7, 639, 191, 4016, 3306, 4474, 21, 5, 2483, 405, 171, 1694, 702, 62, 1202, 109, 272, 122, 482, 1034, 38, 23, 178, 107, 1749, 274, 652, 55, 163, 1171, 305, 58, 8, 2], [0, 2104, 40, 78, 274, 7, 23, 178, 107, 1749, 171, 1694, 702, 168, 642, 9, 27, 995, 286, 2, 2, 110, 10, 745, 171, 1694, 702, 47, 257, 344, 9, 531, 2215, 1647, 345, 8625, 3306, 4474, 7, 639, 191, 4016, 3306, 4474, 21, 5, 2483, 405, 171, 1694, 702, 62, 1202, 109, 272, 122, 482, 1034, 38, 23, 178, 107, 1749, 274, 652, 55, 163, 1171, 305, 58, 8, 1140, 40, 2073, 17, 527, 4, 1013, 577, 832, 745, 16, 100, 180, 109, 45, 1054, 634, 160, 383, 22, 195, 421, 367, 433, 109, 1054, 634, 23, 2], [0, 2104, 40, 

In [30]:
for i, offset in enumerate(inputs["offset_mapping"]):
    print(i, offset)

0 [(0, 0), (0, 2), (2, 3), (3, 10), (10, 15), (15, 19), (19, 20), (20, 21), (21, 22), (22, 28), (28, 30), (30, 34), (34, 37), (37, 41), (41, 45), (45, 48), (48, 50), (50, 54), (54, 55), (0, 0), (0, 0), (0, 4), (4, 8), (8, 10), (10, 14), (14, 17), (17, 19), (19, 22), (22, 23), (23, 28), (28, 33), (33, 37), (37, 38), (38, 41), (41, 44), (44, 47), (47, 48), (48, 51), (51, 55), (55, 59), (59, 64), (64, 68), (68, 71), (71, 77), (77, 82), (82, 86), (86, 91), (91, 92), (92, 96), (96, 99), (99, 103), (103, 108), (108, 110), (110, 114), (114, 117), (117, 122), (122, 126), (126, 130), (130, 133), (133, 138), (138, 144), (144, 147), (147, 148), (148, 151), (151, 154), (154, 157), (157, 161), (161, 166), (166, 170), (170, 173), (173, 176), (176, 179), (179, 183), (183, 184), (184, 189), (189, 193), (193, 195), (195, 199), (199, 202), (202, 207), (207, 212), (212, 216), (216, 220), (220, 226), (226, 231), (231, 236), (236, 239), (239, 240), (240, 241), (241, 242), (242, 248), (248, 253), (253, 258)

`sequence_ids` similar to Next Sentence Prediction

In [31]:
inputs.sequence_ids(0)

[None,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 None,
 None,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 None]

In [32]:
inputs["overflow_to_sample_mapping"][0]

0

In [26]:
answers = raw_datasets["train"][2:5]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([92, 64, 36, 0, 0, 0, 93, 59, 25, 0, 0, 0, 0, 0, 0, 0, 66, 39],
 [93, 65, 37, 0, 0, 0, 94, 60, 26, 0, 0, 0, 0, 0, 0, 0, 77, 50])

In [33]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: dưới nước, labels give: dưới nước


Now let’s check index 4, where we set the labels to (0, 0), which means the answer is not in the context chunk of that feature

In [36]:
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: dưới nước, decoded example: <s> Môi trường sống của ấu trùng lưỡng cư chủ yếu là ở đâu?</s></s> dễ gặp nguy hiểm. Da được dùng như cơ quan hô hấp phụ, một số loài kỳ giông và ếch thiếu phổi phụ thuộc hoàn toàn vào da. Động vật lưỡng cư có hình dáng giống bò sát, nhưng bò sát, cùng với chim và động vật có vú, là các loài động vật có màng ối và không cần có nước để sinh</s>


Indeed, we don’t see the answer inside the context.

In [41]:
max_length = 1024
stride = 128

def preprocess_training_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
         max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [42]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_dataset,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

100%|██████████| 18/18 [01:00<00:00,  3.33s/ba]


(17825, 17825)

In [44]:
inputs.pop("overflow_to_sample_mapping")

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2]

A tiny bit of cleanup of the **offset mappings**. They will contain offsets for the *question* and the *context*, but once we’re in the post-processing stage we won’t have any way to know which part of the input IDs corresponded to the *context* and which part was the *question*. So, we’ll set the offsets corresponding to the *question* to `None`:

In [46]:
def preprocess_validation_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [47]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_dataset,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

100%|██████████| 2/2 [00:22<00:00, 11.03s/ba]


(1415, 1427)

In this case we’ve only added a few samples, so it appears the contexts in the validation dataset are a bit shorter.

In [48]:
small_eval_set = raw_datasets["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

eval_set = small_eval_set.map(
    preprocess_validation_dataset,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

100%|██████████| 1/1 [00:01<00:00,  1.52s/ba]


In [50]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained("vinai/bartpho-syllable").to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

Some weights of MBartForQuestionAnswering were not initialized from the model checkpoint at vinai/bartpho-syllable and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)